In [145]:
#Vizier non single stars: I/357 
#Vizier variables: I/358

In [146]:
import pandas as pd
import numpy as np
import os

In [147]:
os.chdir('/arc/projects/k-pop/catalogues/calsurvey/')

In [148]:
input_survey = 'apogee_dr17_filtered.csv'
output_binaries_survey = 'apogee_dr17_binaries.csv'
output_variability_survey = 'apogee_dr17_variables.csv'
outut_survey = 'apogee_dr17_filtered.parquet'

match_radius = 1.

In [149]:
cds_binaries = 'I/357/tbosb1'
cds_variables = 'I/358/vclassre'

In [150]:
survey = pd.read_csv(input_survey)

In [151]:
survey.head()

,dr3_source_id,origin,origin_id,ra,dec,rv,e_rv,teff,e_teff,logg,e_logg,feh,e_feh,alpha_fe,e_alpha_fe
0,2413929812587459072,origin,b'2M00000019-1924498',0.000832,-19.413851,19.073862,0.038599,5501.7730,25.553564,4.304115,0.024555,-0.27553,0.007253,0.090978,0.007373
1,422596679964513792,origin,b'2M00000032+5737103',0.001335,57.619530,-20.545164,0.065306,6099.7810,45.847330,3.673970,0.030661,-0.25297,0.007646,0.029159,0.011755
2,422596679964513792,origin,b'2M00000032+5737103',0.001335,57.619530,-20.434650,0.062999,6162.0303,41.753815,3.715561,0.025935,-0.21417,0.006151,0.048066,0.009856
3,421077597267551104,origin,b'2M00000068+5710233',0.002850,57.173164,-12.673787,0.029277,5031.2637,10.145057,3.456132,0.023554,-0.16028,0.006890,0.055704,0.006063
4,2772097619417608704,origin,b'2M00000103+1525513',0.004322,15.430942,-12.087924,0.059875,5945.7510,44.246800,4.123192,0.033373,-0.26189,0.008906,0.012409,0.012326


In [152]:
#The files will always have (dr3_source_id, ra, and dec) independent of the survey

In [153]:
#Match with binaries

In [154]:
stilts_command = "stilts cdsskymatch \
                  cdstable=%s  find=best \
                  in=%s ifmt=csv \
                  ra=ra dec=dec radius=%d \
                  out=%s ofmt=csv" %(cds_binaries, input_survey, match_radius, output_binaries_survey)

In [155]:
%%time
os.system(stilts_command)

CPU times: user 1.94 ms, sys: 1.89 ms, total: 3.84 ms
Wall time: 24.3 s


0

In [156]:
#Match with variables

In [157]:
stilts_command = 'stilts cdsskymatch \
                  cdstable=%s  find=best \
                  in=%s ifmt=csv \
                  ra=ra dec=dec radius=%d \
                  out=%s ofmt=csv' %(cds_variables, input_survey, match_radius, output_variability_survey)

In [158]:
%%time
os.system(stilts_command)

CPU times: user 3.45 ms, sys: 3.06 ms, total: 6.51 ms
Wall time: 1min 28s


0

In [160]:
#Read binaries and variables to merge and then perform inverse selection on filtered table

In [161]:
binaries = pd.read_csv(output_binaries_survey)
variables = pd.read_csv(output_variability_survey)

In [162]:
#Drop all the columns except dr3_source_id
binaries = binaries.pop('dr3_source_id')
variables = variables.pop('dr3_source_id')

In [163]:
bin_var = pd.concat([binaries, variables], ignore_index=True)

In [164]:
bin_var.drop_duplicates(keep='first', inplace=True)

In [165]:
f_in1d = np.in1d(survey.dr3_source_id.values, bin_var.values, invert=True)

In [166]:
survey_filtered = survey[f_in1d]

In [167]:
survey_filtered.to_parquet(outut_survey, index=False)

In [168]:
#Remove all the intermediate step catalogues before finish
os.system('rm %s' %output_binaries_survey)
os.system('rm %s' %output_variability_survey)
os.system('rm %s' %input_survey)

0